In [14]:
!pip install mysql-connector-python

In [15]:
import mysql.connector as connector

connection = connector.connect(
    host="localhost",
    user="root",           # o tu usuario
    password="",
    database="LittleLemonDB"
)

print("Conexión exitosa")


Conexión exitosa


In [16]:
# Crear cursor
cursor = connection.cursor()

# Definir la consulta
show_tables_query = "SHOW TABLES"

# Ejecutar la consulta
cursor.execute(show_tables_query)

# Obtener todos los resultados
results = cursor.fetchall()

# Imprimir resultados
print("📋 Tablas en la base de datos:")
for table in results:
    print("-", table[0])


📋 Tablas en la base de datos:
- Bookings
- Customers
- MenuItems
- Menus
- Order_Delivery_Status
- Order_Menu_Items
- Orders
- ordersview
- Staff


In [17]:
# Crear la consulta SQL
join_query = """
SELECT 
    Customers.FullName,
    Customers.ContactNumber,
    Customers.Email,
    Orders.Cost
FROM 
    Customers
JOIN 
    Orders ON Customers.CustomerID = Orders.CustomerID
WHERE 
    Orders.Cost > 60
"""


# Ejecutar la consulta
cursor.execute(join_query)

# Obtener resultados
results = cursor.fetchall()

# Mostrar resultados
print("🧾 Customers with orders over $60:")
for row in results:
    print(f"Name: {row[0]} | Phone: {row[1]} | Email: {row[2]} | Amount: ${row[3]}")


🧾 Customers with orders over $60:
Name: Cliente de prueba | Phone: 1234567890 | Email: cliente@demo.com | Amount: $180.00


In [20]:
# (Re)crea cursor buffered
cursor = connection.cursor(buffered=True)

# 1) GetMaxQuantity
cursor.execute("CALL GetMaxQuantity()")
print("MaxQuantity:", cursor.fetchone()[0])
cursor.nextset()

# 2) AddBooking
cursor.execute("CALL AddBooking(30,1,2,'2025-07-10')")
cursor.nextset()
cursor.execute("SELECT * FROM Bookings WHERE BookingID=30")
print("Booking 30:", cursor.fetchall())
cursor.nextset()

# 3) UpdateBooking
cursor.execute("CALL UpdateBooking(30,'2025-07-11')")
cursor.nextset()
cursor.execute("SELECT BookingDate FROM Bookings WHERE BookingID=30")
print("Updated Date:", cursor.fetchall())
cursor.nextset()

# 4) CancelBooking
cursor.execute("CALL CancelBooking(30)")
cursor.nextset()
cursor.execute("SELECT * FROM Bookings WHERE BookingID=30")
print("After CancelBooking:", cursor.fetchall())
cursor.nextset()

# 5) ManageBooking (éxito)
cursor.execute("CALL ManageBooking(31,2,3,'2025-07-12')")
cursor.nextset()
cursor.execute("SELECT * FROM Bookings WHERE BookingID=31")
print("Booking 31:", cursor.fetchall())
cursor.nextset()

# 5b) ManageBooking (rollback)
cursor.execute("CALL ManageBooking(32,3,3,'2025-07-12')")
cursor.nextset()
cursor.execute("SELECT * FROM Bookings WHERE BookingID=32")
print("Booking 32 (should be empty):", cursor.fetchall())
cursor.nextset()


MaxQuantity: 4
Booking 30: [(30, datetime.date(2025, 7, 10), 2, 1)]
Updated Date: [(datetime.date(2025, 7, 11),)]
After CancelBooking: []
Booking 31: [(31, datetime.date(2025, 7, 12), 3, 2)]
Booking 32 (should be empty): []
